In [1]:
import seaborn as sns
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
sample_df =pd.read_csv("demat.csv")

In C:\Users\admin\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\admin\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [6]:
import scipy.stats as stats

In [7]:
# the below was adapted from:https://www.statology.org/brown-forsythe-test-in-python/
data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
anova_table = pd.DataFrame(data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
anova_table.set_index('Source of Variation', inplace = True)
# calculate SSTR and update anova table
x_bar = sample_df['plasma'].mean()
SSTR = sample_df.groupby('severity').count() * (sample_df.groupby('severity').mean() - x_bar)**2
anova_table['SS']['Between Groups'] = SSTR['plasma'].sum()

# calculate SSE and update anova table
SSE = (sample_df.groupby('severity').count() - 1) * sample_df.groupby('severity').std()**2
anova_table['SS']['Within Groups'] = SSE['plasma'].sum()

# calculate SSTR and update anova table
SSTR = SSTR['plasma'].sum() + SSE['plasma'].sum()
anova_table['SS']['Total'] = SSTR

# update degree of freedom
anova_table['df']['Between Groups'] = sample_df['severity'].nunique() - 1
anova_table['df']['Within Groups'] = sample_df.shape[0] - sample_df['severity'].nunique()
anova_table['df']['Total'] = sample_df.shape[0] - 1

# calculate MS
anova_table['MS'] = anova_table['SS'] / anova_table['df']

# calculate F 
F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
anova_table['F']['Between Groups'] = F

# p-value
anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# F critical 
alpha = 0.05
# possible types "right-tailed, left-tailed, two-tailed"
tail_hypothesis_type = "two-tailed"
if tail_hypothesis_type == "two-tailed":
    alpha /= 2
anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# Final ANOVA Table
anova_table

,SS,df,MS,F,P-value,F crit
Source of Variation,,,,,,
Between Groups,8.98554e+07,3,2.99518e+07,24.5073,1.494e-09,3.42243
Within Groups,5.49971e+07,45,1.22216e+06,,,
Total,1.44852e+08,48,3.01776e+06,,,


In [ ]:
#Based on pvalue of 1.49*10^-9 there is sufficient evidence that there is a difference in means